In [1]:
import sys

In [2]:
sys.path.append('/Users/michael/Documents/GitHub/farkle')

In [3]:
from farkle_sim import analyze_roll

In [4]:
import numpy as np

In [5]:
roll = np.array([1, 1, 1, 5, 5, 5])

In [6]:
analyze_roll.get_scorable(roll)

{'one': 3,
 'five': 3,
 'three-ones': True,
 'three-twos': False,
 'three-threes': False,
 'three-fours': False,
 'three-fives': True,
 'three-sixes': False,
 'four-of-a-kind': False,
 'five-of-a-kind': False,
 'six-of-a-kind': False,
 'straight': False,
 'three-pairs': False,
 'four-and-pair': False,
 'triplets': True}

In [7]:
import configparser

In [8]:
config = configparser.ConfigParser()

In [9]:
config.sections()

[]

In [10]:
config.read('../config/scoring_rules.config')

['../config/scoring_rules.config']

In [11]:
point_dict = {}
for i in config.options('POINTS'):
    point_dict[i] = int(config['POINTS'][i])

In [12]:
point_dict

{'one': 100,
 'five': 50,
 'three-ones': 1000,
 'three-twos': 200,
 'three-threes': 300,
 'three-fours': 400,
 'three-fives': 500,
 'three-sixes': 600,
 'four-of-a-kind': 1000,
 'five-of-a-kind': 2000,
 'six-of-a-kind': 3000,
 'straight': 1500,
 'three-pairs': 1500,
 'four-and-pair': 1500,
 'triplets': 2500}

In [13]:
p_dict = {i : int(config['POINTS'][i]) for i in config.options('POINTS')}

In [14]:
p_dict

{'one': 100,
 'five': 50,
 'three-ones': 1000,
 'three-twos': 200,
 'three-threes': 300,
 'three-fours': 400,
 'three-fives': 500,
 'three-sixes': 600,
 'four-of-a-kind': 1000,
 'five-of-a-kind': 2000,
 'six-of-a-kind': 3000,
 'straight': 1500,
 'three-pairs': 1500,
 'four-and-pair': 1500,
 'triplets': 2500}

In [15]:
roll = np.array([1,1,1,5,5,5])

In [16]:
ar = analyze_roll.get_scorable(roll)

In [17]:
type(ar['triplets'])

numpy.bool_

In [18]:
possible_scores = []
for key in ar:
    if type(ar.get(key)) is np.bool_:
        if ar.get(key) == True:
            possible_scores.append(int(p_dict[key]))
if ar['one'] > 0:
    possible_scores.append(ar['one']*100)
if ar['five'] > 0:
    possible_scores.append(ar['five']*50)

In [19]:
print(possible_scores, max(possible_scores))

[1000, 500, 2500, 300, 150] 2500


In [20]:
ar.get('one')

3

In [21]:
dice_count = {
    'three-ones': 3,
    'three-twos': 3,
    'three-threes': 3,
    'three-fours': 3,
    'three-fives': 3,
    'three-sixes': 3,
    'four-of-a-kind': 4,
    'five-of-a-kind': 5,
    'six-of-a-kind': 6,
    'straight': 6,
    'three-pairs': 6,
    'four-and-pair': 6,
    'triplets': 6}

In [22]:
t = []

In [23]:
len(t)

0

In [24]:
def gather_combos(analyzed_roll):
    three_combo_keys = [
        "three-ones",
        "three-twos",
        "three-threes",
        "three-fours",
        "three-fives",
        "three-sixes",
    ]
    
    hi_combo_keys = [
        "four-of-a-kind",
        "five-of-a-kind", 
        "six-of-a-kind"
    ]
    
    
    full_combo_keys = [
        "six-of-a-kind",
        "straight",
        "three-pairs",
        "four-and-pair",
        "triplets"
    ]
    
    combos = []
    
    for i in full_combo_keys:
        if analyzed_roll[i] == True:
            combos.append(i)
    if len(combos) == 0:
        for i in hi_combo_keys:
            if analyzed_roll[i] == True:
                combos.append(i)
    if len(combos) == 0:
        for i in three_combo_keys:
            if analyzed_roll[i] == True:
                combos.append(i)
                
    return combos

In [25]:
def gather_singles(analyzed_roll):
    single_keys = [
        "one",
        "five"
    ]
    
    singles = []
    
    for i in single_keys:
        # drops if triplet present
        if 3 > analyzed_roll[i] > 0:
            singles.append(i)
    
    return singles

In [26]:
roll = np.array([5,5,5,1,1,1])
ar = analyze_roll.get_scorable(roll)

In [27]:
gather_combos(ar)

['triplets']

In [28]:
gather_singles(ar)

[]

In [29]:
def count_gathered(analyzed_roll):
    gathered_combos = gather_combos(analyzed_roll)
    gathered_singles = gather_singles(analyzed_roll)
    comb_score = []
    sing_score = []
    
    
    if len(gathered_combos)==0 and len(gathered_singles)== 0:
        print("FARKLE")
    

    elif len(gathered_combos) == 0:
        for i in gathered_singles:
            comb_score.append((i, (p_dict[i] * analyzed_roll[i]), analyzed_roll[i]))
        return comb_score

    elif len(gathered_singles) == 0:
        for i in gathered_combos:
            sing_score.append((i, p_dict[i], dice_count[i]))
        return sing_score
    
    else:
        for i in gathered_singles:
            comb_score.append((i, (p_dict[i] * analyzed_roll[i]), analyzed_roll[i]))
        for i in gathered_combos:
            sing_score.append((i, p_dict[i], dice_count[i]))
        return [comb_score, sing_score]

In [30]:
count_gathered(ar)

[('triplets', 2500, 6)]